In [1]:
from google.colab import files
uploaded = files.upload()

Saving Train.tar to Train.tar


In [2]:
#extract tar file
import tarfile
import os

# Create a folder to extract dataset
os.makedirs("trainSet", exist_ok=True)

# Extract Train.tar
with tarfile.open("Train.tar") as tar:
    tar.extractall(path="trainSet")

# Verify class folders inside Train (should be 0 to 42)
print("Folders:", os.listdir("trainSet"))


/tmp/ipython-input-1795300804.py:10: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path="trainSet")


Folders: ['9', '12', '24', '20', '21', '33', '19', '37', '27', '38', '14', '11', '31', '39', '5', '41', '0', '7', '4', '35', '13', '18', '28', '1', '3', '29', '32', '15', '23', '6', '16', '8', '25', '10', '2', '22', '17', '42', '36', '40', '30', '34', '26']


In [3]:
import shutil
import random
import os

# Define the base split folder
base_split_path = "GTSRB"

# Clean (delete) old train/val/test folders if they exist
for split in ['train', 'val', 'test']:
    split_path = os.path.join(base_split_path, split)
    if os.path.exists(split_path):
        shutil.rmtree(split_path)  # Delete the entire folder and its contents
        print(f"🧹 Removed existing folder: {split_path}")


# Create new folders for split data
os.makedirs("GTSRB/train", exist_ok=True)
os.makedirs("GTSRB/val", exist_ok=True)
os.makedirs("GTSRB/test", exist_ok=True)

# Ratios for splitting
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Loop through each class folder (0 to 42)
for class_name in os.listdir("trainSet"):
    class_path = os.path.join("trainSet", class_name)
    if not os.path.isdir(class_path):
        continue

    images = os.listdir(class_path)
    random.shuffle(images)

    n_total = len(images)
    n_train = int(train_ratio * n_total)
    n_val = int(val_ratio * n_total)

    # Make class folders
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join("GTSRB", split, class_name), exist_ok=True)
   # Copy files to corresponding split folders
    for i, img in enumerate(images):
        src = os.path.join(class_path, img)
        if i < n_train:
            dst = os.path.join("GTSRB/train", class_name, img)
        elif i < n_train + n_val:
            dst = os.path.join("GTSRB/val", class_name, img)
        else:
            dst = os.path.join("GTSRB/test", class_name, img)

        shutil.copyfile(src, dst)

print("✅ train/val/test split done.")

# Check sorted folders in each split
for split in ['train', 'val', 'test']:
  print(f"\n📁 {split.upper()} folders:")
  for class_folder in sorted(os.listdir(f"GTSRB/{split}")):
     print(class_folder)


✅ train/val/test split done.

📁 TRAIN folders:
0
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
33
34
35
36
37
38
39
4
40
41
42
5
6
7
8
9

📁 VAL folders:
0
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
33
34
35
36
37
38
39
4
40
41
42
5
6
7
8
9

📁 TEST folders:
0
1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
33
34
35
36
37
38
39
4
40
41
42
5
6
7
8
9


In [4]:
#this cell is use to import library-jessy22/5
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping


In [5]:
#Preprocess the Images
# Path to training directory,validation directory,testing directory
train_dir = "GTSRB/train"
val_dir = "GTSRB/val"
test_dir = "GTSRB/test"

# Image parameters
img_height, img_width = 64, 64 #i set 64*64 pixel so the image not loose too many detail need try n error -jessy22/5
batch_size = 32 #large batch then fasttrain but maybe worse generalization,small batch slow training but better generalization Common values: 16, 32, 64, 128., need try n error-jessy22/5

# Create ImageDataGenerator for rescaling
datagen = ImageDataGenerator(rescale=1./255)# normalize 0-255 to 0-1, faster

# Training data
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,# Number of images per training step
    class_mode='categorical',# Use one-hot encoding for class labels
    shuffle=True# randomly shuffles the order of the training images every epoch.Prevents the model from learning the order of data
)

# Validation data
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',

)

#Test data (for final evaluation)
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 27439 images belonging to 43 classes.
Found 5872 images belonging to 43 classes.
Found 5898 images belonging to 43 classes.
